In [38]:
import gensim 
import logging
import glob, os
import nltk
from nltk import word_tokenize
from pprint import pprint
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import pandas as pd
import numpy as np
#nltk.download('punkt')
#nltk.download('wordnet')

# directory containing all source texts for training the model 
data_dir="/Users/lliu9/populism-hackathon/corpus_populism/United States/2016/Donald Trump/"
os.chdir(data_dir)

lemmatized_corpus = []   
original_corpus = []     

tfdocuments = []
filenames=[]
for filename in glob.glob("*.rtf"):
    filedata = open(filename, 'r').read()
    filenames.append(filename)
    tfdocuments.append(filedata)

print("Number of documents: " + str(len(tfdocuments)))

for xdoc in tfdocuments:               
    tokens = word_tokenize(xdoc)     
    lemmas = [nltk.stem.WordNetLemmatizer().lemmatize(token.lower()) for token in tokens]
    lemmatized_corpus.append(lemmas)    
    original_corpus.append(tokens)    

dictionary = Dictionary(lemmatized_corpus)   # Build the dictionary

# Convert to vector corpus
vectors = [dictionary.doc2bow(text) for text in lemmatized_corpus]

# Build TF-IDF model
tfidf = TfidfModel(vectors)

Number of documents: 73


2019-03-02 16:08:15,903 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-03-02 16:08:16,094 : INFO : built Dictionary(9394 unique tokens: ["''", "'ll", "'m", "'re", "'s"]...) from 73 documents (total 269020 corpus positions)
2019-03-02 16:08:16,264 : INFO : collecting document frequencies
2019-03-02 16:08:16,265 : INFO : PROGRESS: processing document #0
2019-03-02 16:08:16,284 : INFO : calculating IDF weights for 73 documents and 9393 features (63787 matrix non-zeros)


### Jaccard index (Intersection over Union)

In [39]:
def get_jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [43]:
jacsim=pd.DataFrame(index=np.linspace(0,len(tfdocuments)-1,len(tfdocuments)))
for i in np.linspace(0,len(tfdocuments)-1,len(tfdocuments)):
    jacsim[i]=np.nan
for i in range(len(tfdocuments)):
    for j in range(len(tfdocuments)):
        jacsim.iloc[i,j]=100*get_jaccard_sim(tfdocuments[i],tfdocuments[j])
import seaborn as sns
import matplotlib.pylab as plt
plt.subplots(figsize=(300,300))

sns_plot=sns.heatmap(jacsim, annot=True, vmax=30,cmap="YlGnBu",square=False)

In [ ]:
#save the plot
#sns_plot.figure.savefig("output.png")

In [55]:
#Calculate average Jaccard similarities (minus the metrics with itself)
agg=np.mean(jacsim,axis=1)-100/len(tfdocuments)

In [49]:
#Index of max Jaccard similarities
agg.idxmax()

60.0

In [51]:
#File name of max Jaccard similarities
filenames[int(agg.idxmax())]

'Trump102116_2.rtf'

In [52]:
#Index of min Jaccard similarities
agg.idxmin()

45.0

In [54]:
#File name of min Jaccard similarities
filenames[int(agg.idxmin())]

'Trump100316.rtf'